In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from experiment.api import mlflow as mlflow_api

In [2]:
LOG_MODEL_RUN = True

mlflow = mlflow_api.MLFlow()

In [3]:
# Hyperparameters
model_config = {"batch_size": 64, "learning_rate": 0.001, "num_epochs": 2}

# Define a simple feedforward neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(28 * 28, 128),  # Input layer (28x28=784 input features, 128 hidden units)
    nn.ReLU(),  # ReLU activation function
    nn.Linear(128, 64),  # Hidden layer (128 input features, 64 hidden units)
    nn.ReLU(),  # ReLU activation function
    nn.Linear(
        64, 10
    ),  # Output layer (64 input features, 10 output units for 10 classes)
)

# Load MNIST dataset and apply transformations
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, transform=transform, download=True
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=model_config["batch_size"], shuffle=True
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=model_config["learning_rate"])

# Training loop
for epoch in range(model_config["num_epochs"]):
    for images, labels in train_loader:
        images = images.view(-1, 28 * 28)  # Reshape images to flatten them
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    print(f"Epoch [{epoch+1}/{model_config['num_epochs']}], Loss: {loss.item():.4f}")

print("Training finished.")

Epoch [1/2], Loss: 0.3855
Epoch [2/2], Loss: 0.1481
Training finished.


In [6]:
# Load MNIST test dataset
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=model_config['batch_size'], shuffle=False)

# Set model to evaluation mode
model.eval()

correct = 0
total = 0

# Calculate accuracy on the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 28 * 28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get predicted labels
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

# Set model back to training mode
model.train()

Accuracy on the test dataset: 95.03%


Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
)

In [5]:
if LOG_MODEL_RUN:
    # log a model run
    log_dict = {
        "params": model_config,
        "metrics": {
            "accuracy": accuracy,
            "loss": loss.item()
        },
    }

    # extra_artifacts = {
    #     "tokenizer": {
    #         "local_path": tokenizer_path,
    #         "save_path": "data"
    #     }
    # }

    run_id = mlflow.log_experiment_run(
        model=model,
        experiment_name="NLP Experiments",
        run_name=f"RNN: first_run",
        log_dict=log_dict,
        registered_model_name="rnn_experiments",
        # extra_artifacts=extra_artifacts,
        tags={"model": "deep_learning", "library": "pytorch"},
    )

Registered model 'rnn_experiments' already exists. Creating a new version of this model...
2023/09/02 11:35:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: rnn_experiments, version 49
Created version '49' of model 'rnn_experiments'.
